In [1]:
import twodlearn as tdl
import tensorflow as tf
import tensorflow_datasets as tfds
from functools import reduce
#tf.enable_eager_execution()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
#tfds.list_builders()

In [3]:
dataset, info = tfds.load('mnist', with_info=True,
                          split=tfds.Split.TRAIN)
dataset = dataset.shuffle(1024).repeat()\
                 .map(lambda x: (tf.cast(x['image'], tf.float32)/255.0, 
                                 tf.one_hot(x['label'], 10)))\
                 .batch(32).prefetch(tf.data.experimental.AUTOTUNE)
input_shape = dataset.output_shapes[0].as_list()

Instructions for updating:
Colocations handled automatically by placer.


W0310 03:02:25.616345 139801296176960 deprecation.py:323] From /data/marinodl/research/2019/AssociativeGAN/env/lib/python3.6/site-packages/tensorflow/python/ops/control_flow_ops.py:423: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [4]:
model = tdl.stacked.StackedLayers()
model.add(tdl.convnet.Conv2DLayer(
    kernel_size=[5,5], filters=32,
    bias={'trainable': True},
    input_shape=input_shape))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.MaxPool2D(pool_size=[2,2]))
model.add(tdl.convnet.Conv2DLayer(kernel_size=[5,5], filters=64))
model.add(tf.keras.layers.MaxPool2D(pool_size=[2,2]))
model.add(tf.keras.layers.ReLU())
conv_dims = model.compute_output_shape(input_shape=input_shape)
reduce_dims = reduce(lambda x, y: x*y, conv_dims.as_list()[1:])
model.add(tf.keras.layers.Reshape((reduce_dims,)))
model.add(tdl.feedforward.AffineLayer(units=512))
#stacked.add(tf.keras.layers.Dense(units=512))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.Dense(units=10))
model.add(tf.keras.layers.Softmax())

In [5]:
inputs = tf.keras.Input(shape=input_shape[1:])
predictions = model(inputs)
kmodel = tf.keras.Model(inputs=inputs, outputs=predictions)


In [6]:
kmodel.compile(optimizer=tf.train.AdamOptimizer(0.002),
               loss='categorical_crossentropy',
               metrics=['accuracy'])
#tdl.core.variables_initializer(tdl.core.get_variables(stacked))\\
#    .run(session=tf.keras.backend.get_session())

In [7]:
#data.make_initializable_iterator()
iter = dataset.make_one_shot_iterator()

In [8]:
_test = iter.get_next()

In [9]:
kmodel.fit(dataset, epochs=5, steps_per_epoch=info.splits['train'].num_examples//32)

Instructions for updating:
Use tf.cast instead.


W0310 03:02:26.670110 139801296176960 deprecation.py:323] From /data/marinodl/research/2019/AssociativeGAN/env/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1009 - acc: 0.9696
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0414 - acc: 0.9875
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0284 - acc: 0.9912
Epoch 4/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0232 - acc: 0.9931
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0211 - acc: 0.9942


In [9]:
for features in dataset.take(1):
    image, label = features["image"], features["label"]

In [15]:
info.as_proto

name: "mnist"
description: "The MNIST database of handwritten digits."
citation: "@article{lecun2010mnist,\n  title={MNIST handwritten digit database},\n  author={LeCun, Yann and Cortes, Corinna and Burges, CJ},\n  journal={ATT Labs [Online]. Available: http://yann. lecun. com/exdb/mnist},\n  volume={2},\n  year={2010}\n}\n"
location {
  urls: "http://yann.lecun.com/exdb/mnist/"
}
schema {
  feature {
    name: "image"
    type: INT
    shape {
      dim {
        size: 28
      }
      dim {
        size: 28
      }
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "label"
    type: INT
  }
}
splits {
  name: "test"
  num_shards: 1
  statistics {
    num_examples: 10000
    features {
      name: "image"
      num_stats {
        common_stats {
          num_non_missing: 10000
        }
        max: 255.0
      }
    }
    features {
      name: "label"
      num_stats {
        common_stats {
          num_non_missing: 10000
        }
        max: 9.0
      }
    }


In [16]:
info.features

FeaturesDict({'image': Image(shape=(28, 28, 1), dtype=tf.uint8), 'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10)})